# Migrating a User
We added Alice earlier, now we will enable calling

In [ ]:
import os
import requests
import json        # just for pretty printing
from dotenv import load_dotenv

load_dotenv()
WEBEX_TOKEN = os.environ.get("WEBEX_TOKEN", 
                             # replace below if you don't have a .env
                             "NWVkODBmZWItMjM0Yy00ZDRlLWFjZmQtZWE4ZGNmN2Q1ZDVhOTFiY2UyYjQtNTNj_P0A1_0ec1bbe0-b5aa-454a-bb5b-aecc4e6bdb4c")
BASE_URI = "https://webexapis.com/v1"

headers = {
    "Authorization": "Bearer " + WEBEX_TOKEN, 
    "Content-Type": "application/json", 
    "Accept": "application/json"
}

We will assume (basic) users are already present in Control Hub, either added manually or via a directory sync.    
The location is also added in a previous step

In [ ]:
USER = "alice@jseynaev-2xup.wbx.ai"
LOCATION = "BRU-Brussels"
NUMBER = "+3227029911"        # non-existing for demo

### 1. Gather necessary IDs
To migrate a user to Webex Calling, we will need to update licenses and associate the user with a location.   
In our case, all users have a UCM license which will need to be removed, the Webex Calling license needs to be added to the user. For both we will need the corresponding IDs.   
The locationID will also be needed

In [ ]:
LOCATION_ID = ""
UCM_LICENSE_ID = ""
WxC_LICENSE_ID = ""

In [ ]:
try:
    response = requests.get(
        BASE_URI + f"/locations?name={LOCATION}", 
        headers=headers
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    locations = response.json()
    print(json.dumps(locations, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

location = next(l for l in locations["items"] 
             if l["name"] == LOCATION)
LOCATION_ID = location["id"]

In [ ]:
try:
    response = requests.get(
        BASE_URI + f"/licenses", 
        headers=headers
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    licenses = response.json()
    print(json.dumps(licenses, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

# there might be more than one license for the same
WxC_LICENSE_ID = next(
    item for item in licenses["items"] 
    if item["name"] == "Webex Calling - Professional"
)["id"]
UCM_LICENSE_ID = next(
    item for item in licenses["items"]
    if item["name"] == "Unified Communication Manager (UCM)"
)["id"]

In [ ]:
print(WxC_LICENSE_ID)
print("---")
print(UCM_LICENSE_ID)

### 2. Check / add telephone number
For us, users that migrate will already have their own number and it needs to be added in order for it to work with LGW. There are of course other scenario's where a new or other number needs to be set.   
Here, we will:
- pull the list of numbers from the user's location
- check if the user's number is configured and available
- if not configured, we will add the number (if it's not available, throw an error)

In [ ]:
try:
    response = requests.get(
        BASE_URI + f"/telephony/config/numbers?location={LOCATION_ID}", 
        headers=headers
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    numbers = response.json()
    print(json.dumps(numbers, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

In [ ]:
numbers_found = [number for number in numbers["phoneNumbers"] 
       if number["phoneNumber"] == NUMBER]

if numbers_found:
    # we found our number
    print("number configured")
    number = numbers_found[0]

    # check if it's available
    if (number["state"] != "ACTIVE" 
        or number["phoneNumberType"] != "PRIMARY" 
        or number["mainNumber"] 
        or ("owner" in number)):
        print("number is not available for assigning - error")
    else:
        print("number available for assigning")
else:
    print("number not configured")


In [ ]:
if not numbers_found:
    # number is not present, we can add it 
    data = {"phoneNumbers": [NUMBER], "state": "ACTIVE"}
    try:
        response = requests.post(
            BASE_URI + f"/telephony/config/locations/{LOCATION_ID}/numbers",
            headers=headers,
            json=data
        )
        response.raise_for_status()

        print(f"{response.status_code} {response.reason}")
    except requests.exceptions.HTTPError as error:
        print(error)
        print(json.dumps(response.json(), indent=4))

### 3. Update the user (basic)
Now we put it all together. We'll pull the existing user data and update the licenses list as well as the number/location details and then push it back up to activate Webex Calling for the user 


In [25]:
try:
    response = requests.get(
        BASE_URI + "/people", 
        headers=headers,
        params={"email": USER, "callingData": "true"}
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    users = response.json()
    print(json.dumps(users, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

200 OK
{
    "notFoundIds": null,
    "items": [
        {
            "id": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iYjZjNTQ4My00NGNkLTRjOGQtYWNlMS1mZGI4ZGQ1ZmQ4ZDE",
            "emails": [
                "alice@jseynaev-2xup.wbx.ai"
            ],
            "displayName": "Alice",
            "nickName": "Alice",
            "firstName": "Alice",
            "lastName": "Anderson",
            "orgId": "Y2lzY29zcGFyazovL3VzL09SR0FOSVpBVElPTi8wZWMxYmJlMC1iNWFhLTQ1NGEtYmI1Yi1hZWNjNGU2YmRiNGM",
            "roles": [],
            "licenses": [
                "Y2lzY29zcGFyazovL3VzL0xJQ0VOU0UvMGVjMWJiZTAtYjVhYS00NTRhLWJiNWItYWVjYzRlNmJkYjRjOkZNU185ZWNhNzgxNC0zMzEzLTQ2NGYtOTY0Mi0wMjM5ODc1YmM5Zjg",
                "Y2lzY29zcGFyazovL3VzL0xJQ0VOU0UvMGVjMWJiZTAtYjVhYS00NTRhLWJiNWItYWVjYzRlNmJkYjRjOkZTU18xYjcyOGZmOS03ZGU4LTRjYjctOTU0MC0yOTMyMGI1YTQyY2I",
                "Y2lzY29zcGFyazovL3VzL0xJQ0VOU0UvMGVjMWJiZTAtYjVhYS00NTRhLWJiNWItYWVjYzRlNmJkYjRjOkZUQ19hMjQ3MzgyOC1hOTgwLTQ3MmYtODE5ZC02YjljY2U

In [26]:
# the returned response is a list, but as we're searching on email there
# should be only one
user = users["items"][0]

user["locationId"] = LOCATION_ID
user["extension"] = NUMBER[-4:]        # last 4 digits, depends on global config
user["phoneNumbers"] = [               # not needed if gotten from sync(*)
    {"value": NUMBER, "type": "work"}]

if WxC_LICENSE_ID not in user["licenses"]:
    user["licenses"].append(WxC_LICENSE_ID)
if UCM_LICENSE_ID in user["licenses"]:
    user["licenses"].remove(UCM_LICENSE_ID)

try:
    response = requests.put(
        BASE_URI + f"/people/{user['id']}",
        headers=headers,
        params={"callingData": "true"},
        json=user
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(json.dumps(response.json(), indent=4))

200 OK
{
    "id": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iYjZjNTQ4My00NGNkLTRjOGQtYWNlMS1mZGI4ZGQ1ZmQ4ZDE",
    "emails": [
        "alice@jseynaev-2xup.wbx.ai"
    ],
    "phoneNumbers": [
        {
            "type": "work_extension",
            "value": "83229911",
            "primary": true
        },
        {
            "type": "work",
            "value": "+3227029911",
            "primary": true
        }
    ],
    "sipAddresses": [
        {
            "type": "cloud-calling",
            "value": "alice@jseynaev-2xup.calls.webex.com",
            "primary": true
        }
    ],
    "extension": "9911",
    "locationId": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OL2VlYmY5YjhmLTM5YTItNDE3Zi1hNWE4LWI5NzlkYzEyYTEzZQ",
    "displayName": "Alice",
    "nickName": "Alice",
    "firstName": "Alice",
    "lastName": "Anderson",
    "orgId": "Y2lzY29zcGFyazovL3VzL09SR0FOSVpBVElPTi8wZWMxYmJlMC1iNWFhLTQ1NGEtYmI1Yi1hZWNjNGU2YmRiNGM",
    "roles": [],
    "licenses": [
        "Y2lzY29zcGFyazovL3V

(*)
when a user is added by a directory sync and the "work" number is already set, you only need to give the extension and location. The backend will check if the extension matches with the "work" number, so make sure the number in your directory is correct and sync'd properly

### 4. Update user (features)

examples:
- disable voicemail (enabled by default)
- set "number of rings" for voicemail
- set callerID to main number
- international calling ...